In [33]:
import os # To walk through the directories.
import cv2 # OpenCV to read images.
import numpy as np # To manipulate arrays.
import matplotlib.pyplot as plt
from patchify import patchify # To divide the images into smaller patches.
from PIL import Image # To perform cropping or resizing operations in an image.

Images_DSdir = "Dataset/MassachusettsBuildings/Targets"
Targets_DSdir = "Dataset/MassachusettsBuildings/Targets"

In [34]:
Images_DS = []
PatchSize = 256 # To divide the images into 256 patch size.
ImageFiles = os.listdir(Images_DSdir) # Gets a list of all the files in the path.
for ImageFile in ImageFiles:
    ImageCV = cv2.imread(Images_DSdir+'/'+ImageFile, cv2.IMREAD_COLOR)  # Reads each image as BGR.
    ImageCV = cv2.cvtColor(ImageCV,cv2.COLOR_BGR2RGB) # Converts each image as RGB.
    SizeX = (ImageCV.shape[1]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    SizeY = (ImageCV.shape[0]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    ImageCV = Image.fromarray(ImageCV) # Converts the image into a pillow image.
    ImageCV = ImageCV.crop((0, 0, SizeX, SizeY))  # Crops the image from the top left corner.
    ImageCV = np.array(ImageCV) # Converts the image into a numpy array.

    ImagePatches = patchify(ImageCV, (PatchSize, PatchSize, 3), step=PatchSize)  # There is no overlap since the step=PatchSize.
    for i in range(ImagePatches.shape[0]):
        for j in range(ImagePatches.shape[1]):
            ImagePatch = ImagePatches[i,j,:,:] # ImagePatches[i][j][:][:] # Gets each patched image.
            
            # #Use minmaxscaler instead of just dividing by 255. 
            # single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape
            # # single_patch_img = (single_patch_img.astype('float32')) / 255.

            ImagePatch = ImagePatch[0] # Drop the extra unecessary dimension that patchify adds.
            Images_DS.append(ImagePatch) # Adds each patched image into the dataset list.

Images_DS = np.array(Images_DS) # Converts the dataset into a numpy array.
print(Images_DS.shape)

(3750, 256, 256, 3)


In [42]:
Targets_DS = []
PatchSize = 256 # To divide the images into 256 patch size.
TargetFiles = os.listdir(Targets_DSdir) # Gets a list of all the files in the path.
for TargetFile in TargetFiles:
    TargetCV = cv2.imread(Targets_DSdir+'/'+TargetFile, cv2.IMREAD_COLOR)  # Read each image as BGR.
    TargetCV = cv2.cvtColor(TargetCV,cv2.COLOR_BGR2RGB) # Converts each image as RGB.
    SizeX = (TargetCV.shape[1]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    SizeY = (TargetCV.shape[0]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    TargetCV = Image.fromarray(TargetCV) # Converts the image into a pillow image.
    TargetCV = TargetCV.crop((0, 0, SizeX, SizeY))  # Crops the image from the top left corner.
    TargetCV = np.array(TargetCV) # Converts the image into a numpy array.

    TargetPatches = patchify(TargetCV, (PatchSize, PatchSize, 3), step=PatchSize)  # Step = 256 for 256 patches means no overlap
    for i in range(TargetPatches.shape[0]):
        for j in range(TargetPatches.shape[1]):
            TargetPatch = TargetPatches[i,j,:,:] # TargetPatches[i][j][:][:] # Gets each patched image.
            
            # #Use minmaxscaler instead of just dividing by 255. 
            # single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape
            # TargetPatch = TargetPatch.astype("float32")/255.

            TargetPatch = TargetPatch[0] # Drop the extra unecessary dimension that patchify adds.                               
            Targets_DS.append(TargetPatch) # Adds each patched image into the dataset list.

Targets_DS = np.array(Targets_DS) # Converts the dataset into a numpy array.
print(Targets_DS.shape)

(3750, 256, 256, 3)


In [48]:
TargetLables = (Targets_DS[:,:,:,0]/255).astype("int") # Converts into one hot encoded vector (just the first channel, no need for all 3 channels). 
TargetLables = np.array(TargetLables) # Converts the dataset into a numpy array.
TargetLables = np.expand_dims(TargetLables, axis=3) # Expands the dimension at axis 3 to match with the dataset.
print(TargetLables.shape)

(3750, 256, 256, 1)


In [58]:
import torch
import torch.utils.data as data

Dataset = {"feature": torch.tensor([Images_DS], dtype=torch.float32), "label": torch.tensor([TargetLables], dtype=torch.int)} # Merges the images and the targets together.
Train_DS, Test_DS = data.random_split(Dataset, [0.9, 0.1]) # Splits the dataset into a trainset (90%) and a testset (10%).
# Train_DS, Test_DS = data.random_split(Dataset, [0.9, 0.1], generator=torch.Generator().manual_seed(42))
print(len(Train_DS))
print(len(Test_DS))


c:\Users\Tobias Scala\anaconda3\envs\pytorch\Lib\site-packages\torch\utils\data\dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


AttributeError: 'Subset' object has no attribute 'shape'